<a href="https://colab.research.google.com/github/ZKTKZ/deeplearning_from_scratch/blob/master/alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data augs

import random
from PIL import Image
x = random.randint(0,32)
y = random.randint(0,32)
im = Image.open("/content/-2362.7422_292.png")
box = (x,y,80+x,80+y)
im.crop(box)

In [ ]:
trainset = torchvision.datasets.ImageNet(root='./data', split='train', download=None)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128)
testset = torchvision.datasets.ImageNet(root='./data', split='val', download=None)
testloader = torch.utils.data.DataLoader(testset)

In [ ]:
# 1. initialize neural network

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# LRN is slightly different -- n / 5 scaling
# Stride is only specified for 1st conv layer ... assuming for next

class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()
		self.conv1 = nn.Conv2d(3, 96, (11, 11, 3), 4)
		self.conv2 = nn.Conv2d(96, 256, (5,5,48), 4)
		self.conv3 = nn.Conv2d(256, 384, (3,3,256), 4)
		self.conv4 = nn.Conv2d(384, 384, (3,3,192), 4)
		self.conv5 = nn.Conv2d(384, 256, (3,3,192), 4)
		self.fc1 = nn.Linear(256, 4096)
		self.fc2 = nn.Linear(4096, 4096)
		self.fc3 = nn.Linear(4096, 1000)
	
	def forward(self, x):
		x = self.conv1(x)

		x = nn.LocalResponseNorm(x, size=5,k=2)
		x = F.max_pool2d(x, 3, 2)
		x = F.relu(x)

		x = self.conv2(x)
		x = nn.LocalResponseNorm(x, size=5,k=2)
		x = F.max_pool2d(x, 3, 2)
		x = F.relu(x)

		x = self.conv3(x)
		x = F.relu(x)

		x = self.conv4(x)
		x = F.relu(x)

		x = self.conv5(x)
		x = F.max_pool2d(x, 3, 2)
		x = F.relu(x)

		x = self.fc1(x)
		x = F.dropout(0.5)
		
		x = self.fc2(x)
		x = F.dropout(0.5)
		
		x = self.fc3(x)
		return F.softmax(x)

net = Net()

#ImageNet is non trival to access,
#so training loop was abandoned 